In [ ]:
%load_ext kamu
%import_dataset matches
%import_dataset player-scores
%import_dataset player-scores-total

In [ ]:
%%sql -o scores
select * from `player-scores`
where name='KrabbyPattie'OR name='visper'OR name='ItIsIThomas'

In [ ]:
%%local
import numpy as np

import pandas as pd
import hvplot.pandas, holoviews as hv
name = scores["name"].isin(["KrabbyPattie"])
go = pd.DataFrame(
    data={'total goals': [np.sum(scores[name]["goals"])], "total points": [np.sum(scores[name]["score"])],
          "total shots": [np.sum(scores[name]["shots"])], "total saves": [np.sum(scores[name]["saves"])],
          "total assists": [np.sum(scores[name]["assists"])]})

p2 = hv.Layout(
    scores.hvplot.box("score", by="name", height=400, width=300, title="Players score distribution") + scores[
        name].hvplot.table(columns=["match_id", "score", "goals", "assists", "shots", "saves"], selectable=True,
                           width=500) + go.hvplot.table(width=400, height=100)).cols(2)
p2

In [ ]:
%%sql -o df
select * 
from matches order by event_time desc limit 1

In [ ]:
%%local
import json

from rattletrap import Replay

from classes.process import PreProcess, Video, Visualize


# this script reads the RL replay file and parses it into a json(internally) using rattletrap
# afterwards the necessary data is extracted and then turned into a heatmap, displays the match in 2d; see specific explanation below
# the main player is the player from which the replay was grabbed

proc = PreProcess()
heat = Visualize()

#load the converted json file from dataset
queried_replay = Replay.from_json(json.loads(df.loc[0].replay_json))

#process the json for car/bal location and other required variables
car_positions = proc.get_paths(queried_replay)

#extract relevant data from the prior process into format we use for visualization
car_histories = proc.car_positions_to_histories(car_positions[0])

teamord,player_ids = proc.get_var()

#turn data into heatmaps, stats etc..
visuals = heat.vis(car_histories,scores,teamord,player_ids)






In [ ]:
%%local 
html5 = Video()
#turn data into a video for upload after event
video = html5.vid(car_histories)